# Custom Data and Plots

## Setup

In [1]:
import json
import os

import util
import plotly.graph_objects as go

DATA_DIR = "../temp"
MONTHS, YEARS = util.get_months_years(DATA_DIR)

def load(data_name):
    with open(os.path.join("..", "assets", "data", data_name)) as f:
        return json.load(f)
    
def get_default_plot_layout(plot_title, unit):
    return {
        "font": {"family": "Times", "size": 15},
        "paper_bgcolor": "#dfdfdf",
        "plot_bgcolor": "#dfdfdf",
        "margin": {"l": 55, "r": 55, "b": 55, "t": 55},
        "title": {"text": plot_title, "x": 0.5},
        "xaxis": {"title": {"text": "time"}},
        "yaxis": {"title": {"text": unit}, "rangemode": "tozero"},
        "autosize": False,
        "width": 1000,
        "height": 500,
        "xaxis": {"showgrid":True, "gridcolor": "#d1d1d1", "zerolinecolor": "#d1d1d1"},
        "yaxis": {"showgrid":True, "gridcolor": "#d1d1d1", "zerolinecolor": "#d1d1d1"}
    }

## aggregate custom data

In [2]:
ddf = util.load_ddf(DATA_DIR, "general", ("month_index", "year_index", "edits", "user_index", "pos_x", "pos_y"))
ddf

,month_index,year_index,edits,user_index,pos_x,pos_y
npartitions=242,,,,,,
,category[known],uint8,uint32,uint32,int16,int16
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [3]:
# for more examples queries check out "parquet_to_json_stats.py"
monthly_edits = ddf.groupby(["month_index"])["edits"].sum().compute()
monthly_edits

month_index
0             40
1           9023
2           7583
3          20407
4           8587
         ...    
211    111148613
212     96695730
213    114529918
214    119291773
215    105918658
Name: edits, Length: 216, dtype: uint32

In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=MONTHS[monthly_edits.index.values.astype(int)], y=monthly_edits.values))
fig.update_layout(get_default_plot_layout("monthly edit count", "edits"))
fig.show()

## create custom plots with saved data

In [5]:
data = load("created_by_top_100_contributor_count_monthly.json")
selected_mobile_editors = ["MAPS.ME android", "MAPS.ME ios", "StreetComplete", "OsmAnd", "Vespucci", "Go Map!!", "Organic Maps android"]
fig = go.Figure()
for name in selected_mobile_editors:
    fig.add_trace(go.Scatter(x=data["x"], y=data["y_list"][data["y_names"].index(name)], name=name))
fig.update_layout(get_default_plot_layout("monthly contributor count per selected editing apps", "contributors"))
fig.show()